In [3]:
import numpy as np
import pandas as pd
import re

In [184]:
train = pd.read_csv("./Dataset/DT_titanic/Train.csv")
test = pd.read_csv("./Dataset/DT_titanic/Test.csv")

In [185]:
train

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3.0,0.0,"O'Donoghue, Ms. Bridget",female,NaN,0.0,0.0,364856,7.7500,NaN,Q,NaN,NaN,NaN
1,2.0,0.0,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0.0,0.0,250655,26.0000,NaN,S,NaN,NaN,NaN
2,2.0,1.0,"Smith, Miss. Marion Elsie",female,40.0,0.0,0.0,31418,13.0000,NaN,S,9,NaN,NaN
3,3.0,1.0,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",female,31.0,1.0,1.0,363291,20.5250,NaN,S,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,"McCoy, Miss. Agnes",female,NaN,2.0,0.0,367226,23.2500,NaN,Q,16,NaN,NaN
5,2.0,0.0,"Gaskell, Mr. Alfred",male,16.0,0.0,0.0,239865,26.0000,NaN,S,NaN,NaN,"Liverpool / Montreal, PQ"
6,2.0,0.0,"Phillips, Mr. Escott Robert",male,43.0,0.0,1.0,S.O./P.P. 2,21.0000,NaN,S,NaN,NaN,"Ilfracombe, Devon"
7,1.0,1.0,"Leader, Dr. Alice (Farnham)",female,49.0,0.0,0.0,17465,25.9292,D17,S,8,NaN,"New York, NY"
8,1.0,0.0,"Brandeis, Mr. Emil",male,48.0,0.0,0.0,PC 17591,50.4958,B10,C,NaN,208.0,"Omaha, NE"
9,2.0,0.0,"Wheeler, Mr. Edwin ""Frederick""",male,NaN,0.0,0.0,SC/PARIS 2159,12.8750,NaN,S,NaN,NaN,NaN


In [186]:
columns_toRemove = ["ticket","home.dest","boat","body"]
train = train.drop(columns=columns_toRemove)
test = test.drop(columns=columns_toRemove)

In [187]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 10 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
name        1009 non-null object
sex         1009 non-null object
age         812 non-null float64
sibsp       1009 non-null float64
parch       1009 non-null float64
fare        1008 non-null float64
cabin       229 non-null object
embarked    1008 non-null object
dtypes: float64(6), object(4)
memory usage: 78.9+ KB


In [188]:
from sklearn.preprocessing import LabelEncoder

In [189]:

train["age"] = train["age"].fillna(train["age"].mean())
train["fare"] = train["fare"].fillna(train["fare"].mean())
train['embarked'] = train['embarked'].fillna("S")

test["age"] = test["age"].fillna(test["age"].mean())
test["fare"] = test["fare"].fillna(test["fare"].mean())
test['embarked'] = test['embarked'].fillna("S")


In [190]:
le = LabelEncoder()
train["sex"] = le.fit_transform(train["sex"])
train["embarked"] = le.fit_transform(train["embarked"])

test["sex"] = le.fit_transform(test["sex"])
test["embarked"] = le.fit_transform(test["embarked"])


In [191]:
train['relatives'] = train['sibsp'] + train['parch']
train.loc[train['relatives'] > 0, 'not_alone'] = 0
train.loc[train['relatives'] == 0, 'not_alone'] = 1
train['not_alone'] = train['not_alone'].astype(int)

test['relatives'] = train['sibsp'] + test['parch']
test.loc[test['relatives'] > 0, 'not_alone'] = 0
test.loc[test['relatives'] == 0, 'not_alone'] = 1
test['not_alone'] = test['not_alone'].astype(int)

In [192]:


deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

train['cabin'] = train['cabin'].fillna("U0")
train['deck'] = train['cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
train['deck'] = train['deck'].map(deck)
train['deck'] = train['deck'].fillna(0)
train['deck'] = train['deck'].astype(int)
train = train.drop(['cabin'],axis = 1)

test['cabin'] = test['cabin'].fillna("U0")
test['deck'] = test['cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
test['deck'] = test['deck'].map(deck)
test['deck'] = test['deck'].fillna(0)
test['deck'] = test['deck'].astype(int)
# we can now drop the cabin feature
test = test.drop(['cabin'],axis = 1)

train.info()
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 12 columns):
pclass       1009 non-null float64
survived     1009 non-null float64
name         1009 non-null object
sex          1009 non-null int32
age          1009 non-null float64
sibsp        1009 non-null float64
parch        1009 non-null float64
fare         1009 non-null float64
embarked     1009 non-null int32
relatives    1009 non-null float64
not_alone    1009 non-null int32
deck         1009 non-null int32
dtypes: float64(7), int32(4), object(1)
memory usage: 78.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 11 columns):
pclass       300 non-null float64
name         300 non-null object
sex          300 non-null int32
age          300 non-null float64
sibsp        300 non-null float64
parch        300 non-null float64
fare         300 non-null float64
embarked     300 non-null int32
relatives    300 non-null float64
not_alone    

In [193]:
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}


# extract titles
train['title'] = train.name.str.extract(' ([A-Za-z]+)\.', expand=False)
# replace titles with a more common title or as Rare
train['title'] = train['title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                        'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['title'] = train['title'].replace('Mlle', 'Miss')
train['title'] = train['title'].replace('Ms', 'Miss')
train['title'] = train['title'].replace('Mme', 'Mrs')
# convert titles into numbers
train['title'] = train['title'].map(titles)
# filling NaN with 0, to get safe
train['title'] = train['title'].fillna(0)
train = train.drop(['name'], axis=1)




# extract titles
test['title'] = test.name.str.extract(' ([A-Za-z]+)\.', expand=False)
# replace titles with a more common title or as Rare
test['title'] = test['title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                        'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test['title'] = test['title'].replace('Mlle', 'Miss')
test['title'] = test['title'].replace('Ms', 'Miss')
test['title'] = test['title'].replace('Mme', 'Mrs')
# convert titles into numbers
test['title'] = test['title'].map(titles)
# filling NaN with 0, to get safe
test['title'] = test['title'].fillna(0)
test = test.drop(['name'], axis=1)

In [194]:

train['age'] = train['age'].astype(int)
train.loc[ train['age'] <= 11, 'Age'] = 0
train.loc[(train['age'] > 11) & (train['age'] <= 18), 'age'] = 1
train.loc[(train['age'] > 18) & (train['age'] <= 22), 'age'] = 2
train.loc[(train['age'] > 22) & (train['age'] <= 27), 'age'] = 3
train.loc[(train['age'] > 27) & (train['age'] <= 33), 'age'] = 4
train.loc[(train['age'] > 33) & (train['age'] <= 40), 'age'] = 5
train.loc[(train['age'] > 40) & (train['age'] <= 66), 'age'] = 6
train.loc[ train['age'] > 66, 'age'] = 6


test['age'] = test['age'].astype(int)
test.loc[ test['age'] <= 11, 'Age'] = 0
test.loc[(test['age'] > 11) & (test['age'] <= 18), 'age'] = 1
test.loc[(test['age'] > 18) & (test['age'] <= 22), 'age'] = 2
test.loc[(test['age'] > 22) & (test['age'] <= 27), 'age'] = 3
test.loc[(test['age'] > 27) & (test['age'] <= 33), 'age'] = 4
test.loc[(test['age'] > 33) & (test['age'] <= 40), 'age'] = 5
test.loc[(test['age'] > 40) & (test['age'] <= 66), 'age'] = 6
test.loc[ test['age'] > 66, 'age'] = 6



In [195]:
train.loc[ train['fare'] <= 7.91, 'fare'] = 0
train.loc[(train['fare'] > 7.91) & (train['fare'] <= 14.454), 'fare'] = 1
train.loc[(train['fare'] > 14.454) & (train['fare'] <= 31), 'fare']   = 2
train.loc[(train['fare'] > 31) & (train['fare'] <= 99), 'fare']   = 3
train.loc[(train['fare'] > 99) & (train['fare'] <= 250), 'fare']   = 4
train.loc[ train['fare'] > 250, 'fare'] = 5
train['fare'] = train['fare'].astype(int)


test.loc[ test['fare'] <= 7.91, 'fare'] = 0
test.loc[(test['fare'] > 7.91) & (test['fare'] <= 14.454), 'fare'] = 1
test.loc[(test['fare'] > 14.454) & (test['fare'] <= 31), 'fare']   = 2
test.loc[(test['fare'] > 31) & (test['fare'] <= 99), 'fare']   = 3
test.loc[(test['fare'] > 99) & (test['fare'] <= 250), 'fare']   = 4
test.loc[ test['fare'] > 250, 'fare'] = 5
test['fare'] = test['fare'].astype(int)


In [196]:
train['Age_Class']= train['age']* train['pclass']
test['Age_Class']= test['age']* test['pclass']



In [197]:
train['Fare_Per_Person'] = train['fare']/(train['relatives']+1)
train['Fare_Per_Person'] = train['Fare_Per_Person'].astype(int)


test['Fare_Per_Person'] = test['fare']/(test['relatives']+1)
test['Fare_Per_Person'] = test['Fare_Per_Person'].astype(int)
train.head(10)
test.head(10)

,pclass,sex,age,sibsp,parch,fare,embarked,relatives,not_alone,deck,title,Age,Age_Class,Fare_Per_Person
0,1.0,1,5,0.0,0.0,2,2,0.0,1,5,1,NaN,5.0,2
1,3.0,0,4,8.0,2.0,3,2,2.0,0,8,2,NaN,12.0,1
2,1.0,1,4,0.0,0.0,3,2,0.0,1,1,1,NaN,4.0,3
3,2.0,1,5,0.0,0.0,1,2,1.0,0,8,1,NaN,10.0,0
4,2.0,1,4,0.0,0.0,1,2,2.0,0,8,1,NaN,8.0,0
5,2.0,0,4,1.0,1.0,2,2,1.0,0,8,3,NaN,8.0,1
6,1.0,1,5,0.0,0.0,2,0,0.0,1,8,5,NaN,5.0,2
7,3.0,1,4,0.0,0.0,0,0,0.0,1,8,1,NaN,12.0,0
8,3.0,1,2,0.0,0.0,1,2,0.0,1,8,1,NaN,6.0,1
9,1.0,1,4,0.0,0.0,3,2,0.0,1,8,1,NaN,4.0,3


In [208]:
usefulColumns = ["pclass","age","sex","sibsp","fare","parch","embarked","deck","title",\
                 "Age_Class","relatives","not_alone","Fare_Per_Person"]
X_data = pd.DataFrame(train,columns=usefulColumns)
Y_data = pd.DataFrame(train,columns=["survived"])

In [199]:
X_data.head()

,pclass,age,sex,sibsp,fare,parch,embarked,deck,title,Age_Class,relative,not_alone,Fare_Per_Person
0,3.0,4,0,0.0,0,0.0,1,8,2,12.0,NaN,1,0
1,2.0,5,1,0.0,2,0.0,2,8,1,10.0,NaN,1,2
2,2.0,5,0,0.0,1,0.0,2,8,2,10.0,NaN,1,1
3,3.0,4,0,1.0,2,1.0,2,8,3,12.0,NaN,0,0
4,3.0,4,0,2.0,2,0.0,1,8,2,12.0,NaN,0,0


In [200]:
def entropy(col):
    counts = np.unique(col,return_counts=True)
    N = float(col.shape[0])
    entr = 0.0
    for ix in counts[1]:
        p = ix/N
        entr += -(p*np.log2(p))
    return entr

In [201]:
X_data.loc[1]

pclass              2.0
age                 5.0
sex                 1.0
sibsp               0.0
fare                2.0
parch               0.0
embarked            2.0
deck                8.0
title               1.0
Age_Class          10.0
relative            NaN
not_alone           1.0
Fare_Per_Person     2.0
Name: 1, dtype: float64

In [202]:
def data_split(X_data,fkey,fval):
    right = pd.DataFrame([],columns=X_data.columns)
    left = pd.DataFrame([],columns=X_data.columns)
    
    for ix in range(X_data.shape[0]):
        if float(X_data[fkey].loc[ix])>fval:
            right = right.append(X_data.loc[ix])
        else:
            left = left.append(X_data.loc[ix])
            
    return left,right

In [203]:
def info_gain(X_data,fkey,fval):
    left,right = data_split(X_data,fkey,fval)
    
    lw = left.shape[0]/X_data.shape[0]
    rw = right.shape[0]/X_data.shape[0]
    
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -1000000
    else:
        i_gain = entropy(X_data.survived) - lw*entropy(left.survived) - rw*entropy(right.survived)
        
    return i_gain

In [204]:
left,right = data_split(train,"sex",0.5)

In [205]:
left.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked,relatives,not_alone,deck,title,Age,Age_Class,Fare_Per_Person
0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,1.0,8.0,2.0,NaN,12.0,0.0
2,2.0,1.0,0.0,5.0,0.0,0.0,1.0,2.0,0.0,1.0,8.0,2.0,NaN,10.0,1.0
3,3.0,1.0,0.0,4.0,1.0,1.0,2.0,2.0,2.0,0.0,8.0,3.0,NaN,12.0,0.0
4,3.0,1.0,0.0,4.0,2.0,0.0,2.0,1.0,2.0,0.0,8.0,2.0,NaN,12.0,0.0
7,1.0,1.0,0.0,6.0,0.0,0.0,2.0,2.0,0.0,1.0,4.0,5.0,NaN,6.0,2.0


In [206]:
right.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked,relatives,not_alone,deck,title,Age,Age_Class,Fare_Per_Person
1,2.0,0.0,1.0,5.0,0.0,0.0,2.0,2.0,0.0,1.0,8.0,1.0,NaN,10.0,2.0
5,2.0,0.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,1.0,8.0,1.0,NaN,2.0,2.0
6,2.0,0.0,1.0,6.0,0.0,1.0,2.0,2.0,1.0,0.0,8.0,1.0,NaN,12.0,1.0
8,1.0,0.0,1.0,6.0,0.0,0.0,3.0,0.0,0.0,1.0,2.0,1.0,NaN,6.0,3.0
9,2.0,0.0,1.0,4.0,0.0,0.0,1.0,2.0,0.0,1.0,8.0,1.0,NaN,8.0,1.0


In [209]:
for ix in usefulColumns:
    r = info_gain(train,ix,train[ix].mean())
    print(ix,r)

pclass 0.055456910002982474
age 0.001098641672460765
sex 0.19274737190850932
sibsp 0.006492394392888956
fare 0.043519315723333674
parch 0.019756080122948216
embarked 0.0174621386110001
deck 0.0680669514315081
title 0.20257747117554592
Age_Class 0.04327412673690312
relatives 0.025373413951335244
not_alone 0.0253734139513353
Fare_Per_Person 0.008963798978198989


In [210]:
class decisionTree:
    
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.depth = depth
        self.max_depth = max_depth
        self.target = None
    
    
    def train(self,X_train):
        features = ["pclass","age","sex","sibsp","fare","parch","embarked","deck",\
                    "title","Age_Class","relatives","not_alone","Fare_Per_Person"]
        info_gain_list = []
        
        for ix in features:
            i_gain = info_gain(X_train,ix,X_train[ix].mean())
            info_gain_list.append(i_gain)
            
        self.fkey = features[np.argmax(info_gain_list)]
        self.fval = X_train[self.fkey].mean()
        #print(self.fkey)
        
        X_left,X_right = data_split(X_train,self.fkey,self.fval)
        X_left = X_left.reset_index(drop = True)
        X_right = X_right.reset_index(drop = True)
        
        
        if X_left.shape[0] == 0 or X_right.shape[0] == 0:
            if X_train.survived.mean() >= 0.5:
                self.target = 1
            else:
                self.target = 0
            return
        
        if self.depth > self.max_depth:
            if X_train.survived.mean() >= 0.5:
                self.target = 1
            else:
                self.target = 0
            return
        
        self.left = decisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(X_left)
        
        self.right = decisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(X_right)
        
        if X_train.survived.mean() >= 0.5:
            self.target = 1
        else:
            self.target = 0
        return
    
    
    def prediction(self,test):
        if test[self.fkey] > self.fval:
            if self.right is None:
                return self.target
            else:
                return self.right.prediction(test)
        else:
            if self.left is None:
                return self.target
            else:
                return self.left.prediction(test)

In [211]:
d = decisionTree()
d.train(train)

In [212]:
y_pred = []
for i in range(test.shape[0]):
    y_pred.append(d.prediction(test.loc[i]))

In [213]:
y =  pd.DataFrame(data=y_pred,columns=["survived"]) 
y.to_csv("titanic2.csv",index = True)